## Import the required libraries

In [1]:
#Import Libraries
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import chardet
import datetime
from dateutil.relativedelta import relativedelta
from datetime import timedelta
from IPython.core.display import HTML
from datetime import date

## Read the list of files in directory

In [3]:
path = "C:/Users/rika.pajrin/OneDrive - shl-group.com/Documents/QA_Complaint/Complaint Registration/Affected Lot Information/Affected Lot 2023/"
files = os.listdir(path)
print("Files and directories in '", path, "' :")

print(files)

Files and directories in ' C:/Users/rika.pajrin/OneDrive - shl-group.com/Documents/QA_Complaint/Complaint Registration/Affected Lot Information/Affected Lot 2023/ ' :
['BMBC_20210101-20210401.XLSX', 'BMBC_20210402-20210630.XLSX', 'BMBC_20210701-20210930.XLSX', 'BMBC_20211001-20211231.XLSX', 'BMBC_20220101-20220331.XLSX', 'BMBC_20220401-20220630.XLSX', 'BMBC_20220701-20220930.XLSX', 'BMBC_20221001-20221231.XLSX', 'BMBC_20230101-20230331.XLSX', 'COOIS_20210101-20210331.XLSX', 'COOIS_20210401-20210630.XLSX', 'COOIS_20210701-20210930.XLSX', 'COOIS_20211001-20211231.XLSX', 'COOIS_20220101-20220331.XLSX', 'COOIS_20220401-20220630.XLSX', 'COOIS_20220701-20220930.XLSX', 'COOIS_20221001-20221231.XLSX', 'COOIS_20230101-20230331.XLSX']


## Separate the BMBC files and COOS files

In [4]:
files_bmbc = [f for f in files if f[:4] == 'BMBC']
files_coois = [f for f in files if f[:5] == 'COOIS']

## Data manipulation for BMBC files

In [5]:
## append or concat all the BMBC files into one BMBC dataframe
df_bmbc = pd.concat([pd.read_excel(path + file) for file in files_bmbc], ignore_index=True)

In [6]:
## convert the format of column SLED/BBD to not showing the timestamp
df_bmbc['SLED/BBD'] = pd.to_datetime(df_bmbc['SLED/BBD'], errors = 'coerce')

In [7]:
## convert the format of the date to follow the trackwise format
for column in df_bmbc.columns:
    if df_bmbc[column].dtype == 'datetime64[ns]':
        df_bmbc[column] = pd.to_datetime(df_bmbc[column]).dt.strftime('%m/%d/%Y')
        
#df_bmbc.info()

In [31]:
## select the required data column of BMBC
df_bmbc_required = df_bmbc[['Batch','Material Number','Material type','Date of Manuf.','SLED/BBD']]
#df_bmbc_required

## Data manipulation for COOIS files

In [9]:
## append or concat all the coois files
df_coois = pd.concat([pd.read_excel(path + file) for file in files_coois], ignore_index=True)

#df_coois.info()

In [16]:
## remove row if there is null in "Main order number" column
df_coois2 = df_coois.dropna(axis=0, subset=['Main Order Number'])

In [20]:
## select the required data column if COOIS
df_coois_required = df_coois2[['Batch','Material','Material description','Order quantity (GMEIN)','Delivered quantity (GMEIN)']]

In [28]:
## reset the index, and delete the index column
df_coois_required = df_coois_required.reset_index()
del df_coois_required['index']

#df_coois_required.tail(5)

In [33]:
## do vlookup or in pandas way of "inner join", get only the data that consist in both BMBC and COOIS based on batch number
inner_join = pd.merge(df_bmbc_required, 
                      df_coois_required, 
                      on ='Batch', 
                      how ='inner')

In [43]:
#remove the ZSAM material type because it is stock material in warehouse
inner_join_final = inner_join.loc[(inner_join['Material type'] != 'ZSAM')]

#inner_join_final.head(5)

In [45]:
## select the required data for the affected lot excel data 
affected_lot_data = inner_join_final[['Batch','Material Number','Material description','Date of Manuf.','SLED/BBD','Order quantity (GMEIN)','Delivered quantity (GMEIN)']]

In [53]:
## conduct validation to check the data based on the material number
data_validation = affected_lot_data.loc[(affected_lot_data['Material Number'] == '100000017-00')]

data_validation.head(5)

,Batch,Material Number,Material description,Date of Manuf.,SLED/BBD,Order quantity (GMEIN),Delivered quantity (GMEIN)
5396,B000102469,100000017-00,"Slip-On Sleeve,DAI Push Click 1.6",02/25/2021,02/24/2027,42240,42240
10891,B000098279,100000017-00,"Slip-On Sleeve,DAI Push Click 1.6",01/11/2021,01/10/2027,10,10
12609,B000115760,100000017-00,"Slip-On Sleeve,DAI Push Click 1.6",06/28/2021,06/27/2027,95,95
15859,B000112675,100000017-00,"Slip-On Sleeve,DAI Push Click 1.6",06/07/2021,06/06/2027,42240,42240
18964,B000111062,100000017-00,"Slip-On Sleeve,DAI Push Click 1.6",05/17/2021,05/16/2027,42240,42240


In [47]:
## export the data to excel of affected lot information
affected_lot_data.to_excel('Affected Lot information 20210101~20230331.xlsx',index=False)